# Project 1 Portfolio Analyzer Consumer Stocks VS. Tech Stocks

We will be projecting future returns of a portfolio comprised of five consumer stocks as well as a a portfolio comprised of 5 tech stocks over a 15 and 20 year period in order to evaluate the better long term buy. The asset allocation between between stocks and bonds will remain the same between both portfolios.


The Stocks we will be analyzing are 

# Consumer Stocks:             

* P&G
* Johnson and Johnson
* Walmart
* Kellogg
* Coca Cola

# Tech Stocks:

* Amazon 
* Meta
* Google
* Tesla
* NVIDIA



In [1]:
# Importing libraries and dependencies
import pandas as pd
import yfinance as yf
from pathlib import Path

In [4]:
# Importing csv files as a pandas dataframe
# print dataframe
all_stocks = pd.read_csv(Path("Resources/stocks.csv"))
all_stocks       

,Unnamed: 0,Adj Close,Adj Close.1,Adj Close.2,Adj Close.3,Adj Close.4,Adj Close.5,Adj Close.6,Adj Close.7,Adj Close.8,...,Volume,Volume.1,Volume.2,Volume.3,Volume.4,Volume.5,Volume.6,Volume.7,Volume.8,Volume.9
0,NaN,AMZN,GOOGL,JNJ,K,KO,META,NVDA,PG,TSLA,...,AMZN,GOOGL,JNJ,K,KO,META,NVDA,PG,TSLA,WMT
1,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2005-01-03,2.2260000705718994,5.0728230476379395,37.56045913696289,25.440715789794922,11.96491813659668,NaN,1.8030771017074585,33.111019134521484,NaN,...,208930000,633134232,7859500,1327900,14072000,NaN,106671600,4858000,NaN,16078400
3,2005-01-04,2.1070001125335693,4.867366790771484,37.441036224365234,25.34894371032715,11.797857284545898,NaN,1.7181991338729858,32.69705581665039,NaN,...,388370000,549685764,6989600,1263100,16860200,NaN,79022400,5547400,NaN,10596700
4,2005-01-05,2.0885000228881836,4.842593193054199,37.417152404785156,25.503795623779297,11.743133544921875,NaN,1.7342573404312134,33.039039611816406,NaN,...,167084000,329134536,6888100,1546800,13271000,NaN,72481200,7277200,NaN,8241500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4591,2023-03-28,97.23999786376953,101.02999877929688,151.82000732421875,66.29000091552734,61.41999816894531,200.67999267578125,264.1000061035156,146.36000061035156,189.19000244140625,...,38720100,32057900,6115300,1515500,10925700,19127300.0,35610400,4213800,98654600.0,7094300
4592,2023-03-29,100.25,101.38999938964844,153.30999755859375,66.48999786376953,61.86000061035156,205.35000610351562,269.8399963378906,146.80999755859375,193.8800048828125,...,49783300,28779600,6012000,1621800,9441000,18851100.0,39369400,4987700,123660000.0,8072300
4593,2023-03-30,102.0,100.88999938964844,153.42999267578125,66.66000366210938,61.849998474121094,207.83999633789062,273.8299865722656,147.4499969482422,195.27999877929688,...,53633400,33086200,4555700,1466700,9870700,22608300.0,36451600,4117700,110252200.0,7293000
4594,2023-03-31,103.29000091552734,103.7300033569336,155.0,66.95999908447266,62.029998779296875,211.94000244140625,277.7699890136719,148.69000244140625,207.4600067138672,...,56704300,36823200,9888000,1933000,14439400,25412000.0,43324300,7685300,169638500.0,6954400


In [27]:
# cleaning up the data by importing it and setting the header parameter to '1'.
# dropping index parameter '0'
# renaming the dataframe column 'unnamed: 0' to 'Date'
# Print dataframe
all_stocks = pd.read_csv('Resources/stocks.csv', header=1)
all_stocks = all_stocks.drop(all_stocks.index[0])
all_stocks = all_stocks.rename(columns={"Unnamed: 0":"Date"})
all_stocks

,Date,AMZN,GOOGL,JNJ,K,KO,META,NVDA,PG,TSLA,...,AMZN.5,GOOGL.5,JNJ.5,K.5,KO.5,META.5,NVDA.5,PG.5,TSLA.5,WMT.5
1,2005-01-03,2.226000,5.072823,37.560459,25.440716,11.964918,NaN,1.803077,33.111019,NaN,...,208930000.0,633134232.0,7859500.0,1327900.0,14072000.0,NaN,106671600.0,4858000.0,NaN,16078400.0
2,2005-01-04,2.107000,4.867367,37.441036,25.348944,11.797857,NaN,1.718199,32.697056,NaN,...,388370000.0,549685764.0,6989600.0,1263100.0,16860200.0,NaN,79022400.0,5547400.0,NaN,10596700.0
3,2005-01-05,2.088500,4.842593,37.417152,25.503796,11.743134,NaN,1.734257,33.039040,NaN,...,167084000.0,329134536.0,6888100.0,1546800.0,13271000.0,NaN,72481200.0,7277200.0,NaN,8241500.0
4,2005-01-06,2.052500,4.718468,37.524647,25.538210,11.841063,NaN,1.717435,33.201015,NaN,...,174018000.0,415068516.0,5277500.0,1039000.0,10038600.0,NaN,56356800.0,5261200.0,NaN,14028800.0
5,2005-01-07,2.116000,4.851101,37.387299,25.354683,11.855463,NaN,1.684554,33.548973,NaN,...,196732000.0,386129484.0,5037800.0,993800.0,9732800.0,NaN,76309200.0,6250500.0,NaN,9660300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4590,2023-03-28,97.239998,101.029999,151.820007,66.290001,61.419998,200.679993,264.100006,146.360001,189.190002,...,38720100.0,32057900.0,6115300.0,1515500.0,10925700.0,19127300.0,35610400.0,4213800.0,98654600.0,7094300.0
4591,2023-03-29,100.250000,101.389999,153.309998,66.489998,61.860001,205.350006,269.839996,146.809998,193.880005,...,49783300.0,28779600.0,6012000.0,1621800.0,9441000.0,18851100.0,39369400.0,4987700.0,123660000.0,8072300.0
4592,2023-03-30,102.000000,100.889999,153.429993,66.660004,61.849998,207.839996,273.829987,147.449997,195.279999,...,53633400.0,33086200.0,4555700.0,1466700.0,9870700.0,22608300.0,36451600.0,4117700.0,110252200.0,7293000.0
4593,2023-03-31,103.290001,103.730003,155.000000,66.959999,62.029999,211.940002,277.769989,148.690002,207.460007,...,56704300.0,36823200.0,9888000.0,1933000.0,14439400.0,25412000.0,43324300.0,7685300.0,169638500.0,6954400.0
